In [1]:
import pandas as pd
import os
import pickle
import bmra_prep
import bmra_prep.pathway_activity.prediction

In [2]:
data_dir = "/home/jyang/Phd_project/project_GBM/gbm_Scripts/gbm_Scripts_bmra_u251mg/01_outputs_2020"
#info_dir = data_dir

out_dir = "02_outputs_2020"

os.makedirs(out_dir, exist_ok = True)


# Load Data

In [3]:
# load metdadata dict and extract used elements
with open(os.path.join(data_dir, "metadata.pickle"), "rb") as f:
    all_metadata = pickle.load(f)

n_modules = all_metadata["n_modules"]
n_genes = all_metadata["n_genes"]
n_experiments = all_metadata["n_experiments"]

modules = all_metadata["modules"]
exp_ids = all_metadata["exp_ids"]
genes = all_metadata["genes"]

In [4]:
# load data
L1000_df = pd.read_csv(
    os.path.join(data_dir, "L1000_Data_norm_data.csv"),
    index_col = 0,
)

x = L1000_df.values
x.shape

(978, 98)

In [5]:
# load doses and perturbation matrix
inhib_conc_matrix = pd.read_csv(
    os.path.join(data_dir, "inhib_conc_annotated.csv"),
    index_col = 0,
).values

ic50_matrix = pd.read_csv(
    os.path.join(data_dir, "ic50_annotated.csv"),
    index_col = 0,
).values

# gamma_matrix = pd.read_csv(
#     os.path.join(data_dir, "gamma_annotated.csv"),
#     index_col = 0,
# ).values

pert_matrix = pd.read_csv(
    os.path.join(data_dir, "pert_annotated.csv"),
    index_col = 0,
).values

In [6]:
# y_true = (1 + gamma_matrix * inhib_conc_matrix / ic50_matrix) / (1 + inhib_conc_matrix / ic50_matrix)

y_true = 1 / (1 + inhib_conc_matrix / ic50_matrix)

display(y_true.shape)
y_true

(10, 98)

array([[1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
        1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.000000

## Run models

In [7]:
a_coeffs = bmra_prep.pathway_activity.prediction.predict_coeffs(
    x, y_true, pert_matrix, 200_000, 10, 10, 10, 100)

In [8]:
a_coeffs_df = pd.DataFrame(a_coeffs, index = modules, columns = genes)
a_coeffs_df.to_csv(os.path.join(out_dir, "a_coeffs.csv"))
#a_coeffs_df = pd.read_csv(os.path.join(out_dir,'a_coeffs.csv'),index_col=0)
#a_coeffs = a_coeffs_df.values
display(a_coeffs_df.astype(bool).sum(axis='columns'))
display(a_coeffs_df)

CDK1        978
CDK2        978
CDK4_6      978
PI3K        978
ERK         978
PDGFR       978
Aurora      978
Estrogen    978
EGFR        978
p53         978
dtype: int64

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
CDK1,0.000020,0.000007,2.063131e-05,1.233888e-05,0.000001,-0.000020,-1.402300e-05,0.000006,-0.000008,1.419706e-05,...,0.000003,6.589850e-06,-6.729833e-06,0.000003,0.000027,-0.000003,0.000012,0.000004,-0.000024,2.088767e-05
CDK2,0.000013,0.000006,-4.911273e-06,3.293049e-05,-0.000004,-0.000022,2.896950e-05,0.000012,-0.000009,3.403067e-05,...,0.000008,1.988464e-06,-6.771807e-05,-0.000005,-0.000007,-0.000012,-0.000003,0.000010,0.000011,-5.036000e-06
CDK4_6,-0.000015,-0.000014,1.237638e-05,6.841862e-06,-0.000028,0.000007,-2.002427e-05,0.000051,0.000009,-2.065023e-05,...,0.000020,1.060279e-05,2.224549e-05,-0.000011,-0.000014,0.000006,0.000010,0.000005,-0.000002,-2.818501e-06
PI3K,-0.000013,0.000001,-6.630660e-06,4.312501e-06,0.000011,0.000024,1.257144e-05,0.000002,-0.000019,-1.377396e-05,...,0.000005,9.032177e-06,-5.086328e-06,-0.000002,-0.000040,0.000003,0.000021,0.000007,0.000005,4.442749e-07
ERK,-0.000023,0.000004,2.287549e-05,1.265939e-05,0.000002,-0.000038,-2.214677e-05,0.000002,0.000009,4.915816e-06,...,-0.000032,-1.338960e-06,1.978053e-08,0.000008,0.000018,0.178093,0.000016,-0.000014,0.000009,3.593274e-06
PDGFR,-0.000015,-0.000003,1.830272e-05,-5.217089e-06,0.000031,0.000011,-1.520497e-05,0.000004,0.000012,-9.235681e-06,...,0.000007,5.422618e-07,1.456215e-05,-0.000017,-0.000002,0.023222,0.000024,0.000003,0.000020,1.283181e-05
Aurora,0.000018,0.000011,7.492437e-06,3.225417e-07,0.000012,0.000004,-3.364191e-05,0.000006,0.000013,-3.787285e-07,...,-0.000006,-5.052195e-06,-5.861033e-06,0.000008,-0.000008,0.000117,0.000009,0.000008,0.000001,4.783167e-06
Estrogen,0.000002,0.000006,-8.202001e-06,4.526438e-06,0.000014,-0.000005,-1.053839e-05,-0.000024,-0.000015,6.779665e-07,...,-0.000031,-1.461800e-06,1.488899e-05,0.000001,-0.000008,0.001394,0.000012,-0.000015,-0.000006,5.306600e-07
EGFR,-0.000024,-0.000002,1.238237e-05,2.312260e-06,0.000007,0.000002,-6.207704e-07,0.000011,0.174312,-9.938932e-07,...,-0.000015,8.499568e-06,-2.918211e-06,0.000002,-0.000010,0.000020,-0.000011,-0.000048,0.000003,2.484047e-05
p53,0.000017,0.000020,-6.334918e-07,-9.270797e-06,0.000002,0.000011,1.003375e-05,0.000006,0.000019,-9.593716e-06,...,-0.000002,9.756453e-06,1.150497e-05,-0.000011,0.000019,-0.000008,-0.000002,-0.000001,0.000015,-1.313788e-06


In [9]:
#pathway_activity = a_coeffs @ x
#pathway_activity.shape

In [10]:
R_global = bmra_prep.pathway_activity.calc_global_response_from_pathway_activity(
    bmra_prep.pathway_activity.calc_pathway_activity(x,a_coeffs),
    modules,
    L1000_df.columns
)
R_global_df = R_global.dataframe
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:A19,ASG002_U251MG_24H:A20,ASG002_U251MG_24H:A21,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,...,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:O13,ASG002_U251MG_24H:O14,ASG002_U251MG_24H:O15,ASG002_U251MG_24H:O22,ASG002_U251MG_24H:O23,ASG002_U251MG_24H:O24,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK1,0.081358,0.122608,0.173062,-0.614592,0.266150,-0.896698,-0.760904,-0.803477,-0.029663,-0.945355,...,0.286703,0.152635,0.215968,0.297845,0.177967,-0.392996,0.156267,-0.471839,0.189555,-0.487124
CDK2,0.106291,0.117066,0.009879,0.098416,0.053292,0.060298,0.086446,0.066069,0.094024,0.064853,...,0.008412,0.014318,0.021080,0.036217,0.027758,-0.001768,0.047324,0.050914,0.094637,-0.162118
CDK4_6,-0.392729,-0.013477,0.138345,-0.309113,0.001109,-0.597966,-0.383819,-0.248611,-0.059019,-1.643305,...,0.337780,-0.104207,-0.123022,0.450489,-0.049429,0.068921,-0.013531,-1.813971,-1.731328,-1.090763
PI3K,-1.729945,-1.923107,-1.579807,-1.766978,-1.359502,-0.459762,-0.091445,-0.141465,-0.360043,-0.483704,...,-0.553317,-1.046034,0.472570,-1.020177,-1.449277,-1.431845,-0.862150,-1.074738,-1.142248,-0.761477
ERK,-0.056256,-0.910588,-0.579399,-1.369207,-0.237407,-0.074648,-0.052447,0.029301,0.071882,0.243400,...,-0.517928,-1.977276,-1.642834,-1.597361,-1.648355,-1.868634,-1.976509,-0.569702,-1.975754,-0.282942
PDGFR,-1.673658,-0.121245,-0.608819,-0.103241,-0.552716,-0.669133,0.035077,0.067963,0.121428,0.163953,...,-1.118310,-1.519820,-0.415189,-1.327737,-0.535055,-0.053033,-0.715650,-1.857551,-1.000883,-0.340180
Aurora,0.390676,-0.868365,0.130717,0.536593,0.582838,-1.997585,-1.329816,-1.644541,-1.583619,-1.070000,...,0.152283,-1.002509,-0.457189,0.318413,-0.201817,0.633230,0.635859,-0.908544,-1.063241,-0.523690
Estrogen,0.178508,-0.060196,-0.460387,0.059269,-1.570905,-0.186924,-0.070257,0.116651,0.111048,0.154403,...,-1.395481,-0.307969,-0.315300,-0.317254,-0.626320,-0.101909,-1.546800,-0.857482,-0.177886,0.786530
EGFR,-0.247294,-1.110090,0.864011,-1.969531,-0.573473,-1.019544,-0.859004,-1.118882,-1.316839,-1.928089,...,0.996038,-1.176345,0.951608,-0.924067,-1.171091,0.973031,-0.725879,-0.704517,-0.729730,-0.316604
p53,-0.247018,0.238750,0.089139,0.117104,-0.310950,-1.426553,-1.594359,-1.570430,-1.510393,-0.777911,...,0.422952,0.179666,-0.000257,0.425149,0.064065,0.406067,0.383263,0.150027,0.035401,-0.004248


In [11]:
R_global_df.to_csv(os.path.join(out_dir, "R_global_annotated.csv"))
display(R_global_df)

,ASG002_U251MG_24H:A10,ASG002_U251MG_24H:A11,ASG002_U251MG_24H:A12,ASG002_U251MG_24H:A19,ASG002_U251MG_24H:A20,ASG002_U251MG_24H:A21,ASG002_U251MG_24H:B10,ASG002_U251MG_24H:B11,ASG002_U251MG_24H:B12,ASG002_U251MG_24H:B13,...,ASG002_U251MG_24H:O12,ASG002_U251MG_24H:O13,ASG002_U251MG_24H:O14,ASG002_U251MG_24H:O15,ASG002_U251MG_24H:O22,ASG002_U251MG_24H:O23,ASG002_U251MG_24H:O24,ASG002_U251MG_24H:P16,ASG002_U251MG_24H:P17,ASG002_U251MG_24H:P18
CDK1,0.081358,0.122608,0.173062,-0.614592,0.266150,-0.896698,-0.760904,-0.803477,-0.029663,-0.945355,...,0.286703,0.152635,0.215968,0.297845,0.177967,-0.392996,0.156267,-0.471839,0.189555,-0.487124
CDK2,0.106291,0.117066,0.009879,0.098416,0.053292,0.060298,0.086446,0.066069,0.094024,0.064853,...,0.008412,0.014318,0.021080,0.036217,0.027758,-0.001768,0.047324,0.050914,0.094637,-0.162118
CDK4_6,-0.392729,-0.013477,0.138345,-0.309113,0.001109,-0.597966,-0.383819,-0.248611,-0.059019,-1.643305,...,0.337780,-0.104207,-0.123022,0.450489,-0.049429,0.068921,-0.013531,-1.813971,-1.731328,-1.090763
PI3K,-1.729945,-1.923107,-1.579807,-1.766978,-1.359502,-0.459762,-0.091445,-0.141465,-0.360043,-0.483704,...,-0.553317,-1.046034,0.472570,-1.020177,-1.449277,-1.431845,-0.862150,-1.074738,-1.142248,-0.761477
ERK,-0.056256,-0.910588,-0.579399,-1.369207,-0.237407,-0.074648,-0.052447,0.029301,0.071882,0.243400,...,-0.517928,-1.977276,-1.642834,-1.597361,-1.648355,-1.868634,-1.976509,-0.569702,-1.975754,-0.282942
PDGFR,-1.673658,-0.121245,-0.608819,-0.103241,-0.552716,-0.669133,0.035077,0.067963,0.121428,0.163953,...,-1.118310,-1.519820,-0.415189,-1.327737,-0.535055,-0.053033,-0.715650,-1.857551,-1.000883,-0.340180
Aurora,0.390676,-0.868365,0.130717,0.536593,0.582838,-1.997585,-1.329816,-1.644541,-1.583619,-1.070000,...,0.152283,-1.002509,-0.457189,0.318413,-0.201817,0.633230,0.635859,-0.908544,-1.063241,-0.523690
Estrogen,0.178508,-0.060196,-0.460387,0.059269,-1.570905,-0.186924,-0.070257,0.116651,0.111048,0.154403,...,-1.395481,-0.307969,-0.315300,-0.317254,-0.626320,-0.101909,-1.546800,-0.857482,-0.177886,0.786530
EGFR,-0.247294,-1.110090,0.864011,-1.969531,-0.573473,-1.019544,-0.859004,-1.118882,-1.316839,-1.928089,...,0.996038,-1.176345,0.951608,-0.924067,-1.171091,0.973031,-0.725879,-0.704517,-0.729730,-0.316604
p53,-0.247018,0.238750,0.089139,0.117104,-0.310950,-1.426553,-1.594359,-1.570430,-1.510393,-0.777911,...,0.422952,0.179666,-0.000257,0.425149,0.064065,0.406067,0.383263,0.150027,0.035401,-0.004248
